In [1]:
import os
from datetime import datetime
import h5py
import numpy as np
import pandas as pd
from data_helpers import format_timeseries
from tempConv import tempConvDecoder

%load_ext autoreload
%autoreload 2
%matplotlib inline

Using Theano backend.


In [4]:
conf = {
    'bs' : 256,
    'eps' : 25,
    'lr' : 0.0005,
    'kernel' : 2,
    'nb_filter' : 5,
    'window' : 60,
    'offset' : 10,
    'regressor' : False,
    'pyramidal' : True,
    'resample_data' : False,
    'sample_size' : 40000,
    'run_id' : datetime.now(),
    'verbose' : False,
    'key' : ['yaw_vel'],
    'dataset' : None
}

In [5]:
path = 'datasets/GRat31'
X_train, X_test, y_train, y_test = format_timeseries(
    path, 
    conf['window'],
    conf['offset'],
    (0,384,6), 
    (5,6),
    resample_data=conf['resample_data'],
    sample_size=conf['sample_size'],
    regressor=conf['regressor']
)

dataset = {
    'train' : (X_train, y_train),
    'test' : (X_test, y_test)
}

conf['dataset'] = dataset

['636429913515267697.hdf5', '636428029026710180.hdf5', '636430663717571697.hdf5', '636427282621202061.hdf5', '636431765535543697.hdf5', '636438658377315948.hdf5', '636439164041965948.hdf5', '636428953768193973.hdf5']
X_train and y_train shape: (24956, 60, 64) (24956, 1)


In [6]:
stats = []

In [ ]:
TCD = tempConvDecoder(**conf)
TCD.fit()
R2s,rs = TCD.determine_fit()
stats.append([R2s, rs])

WARNING (theano.gof.cmodule): ModuleCache.refresh() Found key without dll in cache, deleting it. /n/home07/dapello/.theano/compiledir_Linux-3.10-el7.x86_64-x86_64-with-centos-7.2.1511-Core-x86_64-3.6.0-64/tmp0yg8p21m/key.pkl
WARNING (theano.gof.compilelock): Overriding existing lock by dead process '11310' (I am process '11761')


In [6]:
print(stats)

[[[-0.0003992840288902233], [-0.011170856030242468]]]


In [44]:
TCD.model.evaluate(TCD.X_test,TCD.y_test)

24608/24913 [============================>.] - ETA: 0s

[0.52790511442393295, 0.74992975554931163]